In [1]:
from dotenv import load_dotenv

load_dotenv("openai.env")

True

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话")
model = ChatOpenAI(temperature=0)
chain = prompt | model

In [8]:
chain.invoke({"topic": "大头"})

AIMessage(content='好的，这里有一个关于大头的笑话：\n\n为什么大头总是很聪明？\n\n因为大脑藏在大头里啊！哈哈哈！', response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 19, 'total_tokens': 66}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-f5a38bed-c237-4c39-8e23-0dc20078d6fb-0')

# 自定义停止输出符

In [6]:
# stop=["\n"] 碰到换行就自动停止输出
chain = prompt | model.bind(stop=["\n"])
chain.invoke({"topic": "大头"})

AIMessage(content='好的，这里有一个关于大头的笑话：', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 19, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-759b0f10-4b6e-48a0-bbcb-ebc9ade2d2f0-0')

# 兼容openai函数调用的方式

In [9]:
functions = [
    {
        "name": "joke",
        "description": "讲笑话",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "笑话的开头"},
                "punchline": {"type": "string", "description": "爆梗的结尾"},
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

In [11]:
chain.invoke({"topic": "男人"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': 'const joke: functions.joke = ({ setup, punchline }) => {\n  console.log(setup);\n  console.log(punchline);\n}\n\njoke({\n  setup: "为什么男人总是喜欢开车？",\n  punchline: "因为开车的时候，他们可以控制方向盘，而不是被女人控制。"\n});', 'name': 'joke'}}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 89, 'total_tokens': 179}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5835f478-2b25-46d2-b2dc-5cbfaa1962af-0')

In [14]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
        prompt
        | model.bind(function_call={"name": "joke"}, functions=functions)
        | JsonOutputFunctionsParser()
)
chain.invoke({"topic": "女人"})

{'setup': '为什么女人喜欢逛街？', 'punchline': '因为那是唯一能让她们感觉自己在控制的地方！'}

# 指定输出字段-只输出setup

In [16]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain_core.runnables import RunnablePassthrough

chain = (
        {"topic": RunnablePassthrough()}  #使用RunnablePassthrough()跳过prompt
        | prompt
        | model.bind(function_call={"name": "joke"}, functions=functions)
        | JsonKeyOutputFunctionsParser(key_name="setup")  # 定义输出的key
)

#chain.invoke({"foo": "女人"})
chain.invoke("女人")

'为什么女人喜欢逛街？'